In [1]:
import os

import pandas as pd

from TokenUsage import showTokenUsage
from dotenv import load_dotenv
from nixtlats import TimeGPT

load_dotenv()

True

In [2]:
timegpt = TimeGPT(token=os.getenv("TIMEGPT_TOKEN"))
timegpt.validate_token()

INFO:nixtlats.timegpt:Happy Forecasting! :), If you have questions or need support, please email ops@nixtla.io


True

In [3]:
showTokenUsage()


Token usage update timed out. There was either no update or the API is down.

TOTAL USAGE: API Calls = 29 | Tokens = 166029 | Spent = 284.28



In [4]:
def dataPrep(dfName, csvPath, timestampCol, dfTempName, dropCol):

    dfName = pd.read_csv(csvPath)

    # Convert TimePeriod column to datetime and remove timezone information
    dfName[timestampCol] = pd.to_datetime(
        dfName[timestampCol]
    ).dt.tz_localize(None)

    # Remove unused column that causes error in TimeGPT detect_anomalies() method
    dfTempName = dfName.drop(columns=[dropCol])

    return dfTempName

In [5]:
def runAndUpdate(dfName, csvPath, timestampCol, dfTempName, dropCol):
    df = dataPrep(dfName, csvPath, timestampCol, dfTempName, dropCol)
    
    anomalies = timegpt.detect_anomalies(
        df,
        time_col=df.columns[0],
        target_col=df.columns[1],
    )
    
    showAnomalies = anomalies[anomalies["anomaly"] == 1]
    showUsage = showTokenUsage()
    return showAnomalies, showUsage

In [6]:
runAndUpdate("billingData", "billing_data_sample.csv", "TimePeriod", "bdTemp", "Tenant")

INFO:nixtlats.timegpt:Validating inputs...
INFO:nixtlats.timegpt:Preprocessing dataframes...
INFO:nixtlats.timegpt:Inferred freq: D
INFO:nixtlats.timegpt:Calling Anomaly Detector Endpoint...



<< Current run: 1 call made | 913 tokens used | 2.57 spent >>

TOTAL USAGE: API Calls = 30 | Tokens = 166942 | Spent = 286.85



(     TimePeriod  anomaly  TimeGPT-lo-99      TimeGPT  TimeGPT-hi-99
 87   2024-01-04        1   1.217690e+06  1487136.000   1.756582e+06
 88   2024-01-05        1   1.197563e+06  1467008.625   1.736454e+06
 103  2024-01-20        1   1.356944e+06  1626389.625   1.895835e+06
 104  2024-01-21        1   1.250522e+06  1519967.250   1.789413e+06
 129  2024-02-15        1   1.851312e+06  2120758.000   2.390204e+06,
 None)